# Day 5 - Lab 1: Tool-Using Agents

**Objective:** Build agents that can use external tools to accomplish tasks they cannot perform on their own, using both LangChain and custom Python functions.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 5! We are now shifting from using AI to build a traditional application to building applications that *are* AI. An 'agent' is more than just a prompt; it's a system that can reason, plan, and use tools to achieve a goal. In this lab, you will build your first simple agents using the flexible LangChain framework.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will set up our environment, which for today includes installing the `langchain`, `langchain-community`, and `tavily-python` libraries. Tavily is a search engine API optimized for AI agents, which we will use as our first tool.

**Model Selection:**
For agentic tasks that require reasoning and tool use, highly capable models are recommended. `gpt-4.1`, `o3`, `gemini-2.5-pro`, or open-source models like `deepseek-ai/DeepSeek-R1` are excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM (though we'll mostly use LangChain's abstractions).

In [8]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# This helper will install packages if they are not found
import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        # Note: %pip is for notebooks. In a script, use subprocess.
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('tavily')

from utils import setup_llm_client
client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

✅ LLM Client configured: Using 'openai' with model 'gpt-4.1'


## Step 2: The Challenges

### Challenge 1 (Foundational): Building a LangChain Agent with One Tool

**Task:** Build a simple LangChain agent that can use the Tavily Search API to answer questions about current events.

**Instructions:**
1.  Import `TavilySearchResults` from `langchain_community.tools.tavily_search`.
2.  Import `create_tool_calling_agent` and `AgentExecutor` from `langchain.agents`.
3.  Import `ChatPromptTemplate` from `langchain_core.prompts`.
4.  Instantiate the `TavilySearchResults` tool.
5.  Create a prompt template. It must include a placeholder for `agent_scratchpad`.
6.  Create the agent by passing the LLM, the list of tools, and the prompt to `create_tool_calling_agent`.
7.  Create an `AgentExecutor` to run the agent.
8.  Invoke the agent with a question that requires a web search, like "What was the score of the last Super Bowl?"

**Expected Quality:** The agent should successfully use the Tavily tool to search the web, find the correct information, and provide an accurate answer.

In [9]:
# TODO: Perform all necessary imports
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# We need to use a LangChain LLM wrapper
llm = ChatOpenAI(model=model_name)

# TODO: 1. Instantiate the Tavily search tool
search_tool = TavilySearchResults(
    client=client,
    model_name=model_name,
    api_provider=api_provider,
)
tools = [search_tool]

# TODO: 2. Create the prompt template
# Make sure to include placeholders for 'input' and 'agent_scratchpad'
prompt = ChatPromptTemplate.from_messages( 
    [
        ("system", "You are a helpful assistant that answers questions using search results."),
        ("human", "{input}"),
        ("ai", "Here are the search results: {agent_scratchpad}"),
    ]
)

# TODO: 3. Create the agent
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

# TODO: 4. Create the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

# TODO: 5. Invoke the agent with a question
question = "What was the score of the last Super Bowl?"
result = agent_executor.invoke({"input": question})

print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'last Super Bowl score'}`


[{'title': 'Final score: Eagles defeat Chiefs 40-22 in Super Bowl LIX', 'url': 'https://sports.yahoo.com/final-score-eagles-defeat-chiefs-034125398.html', 'content': "Published Time: 2025-02-10T03:41:25.000Z\n\nFinal score: Eagles defeat Chiefs 40-22 in Super Bowl LIX - Yahoo Sports\n===============\n\n February 10, 2025\n\nDespite the Chiefs' generally poor performance in the championship game, there were still a few bright spots that Kansas City's fans can take solace in as the team heads into the offseason.\n\nAdvertisement\n\nAdvertisement\n\nAdvertisement [...] Terms and Privacy Policy\n    \n   Privacy Dashboard\n   Help\n   Share Your Feedback\n   About Us\n   About Our Ads\n\nFollow us on\n\n   [](\n   [](\n   [](\n   [](\n   [](\n   [](\n\nDownload our app\n\n   Image 15: Download app from appStore\n   Image 16: Download app from googlePlay\n\n© 2025 Yaho

### Challenge 2 (Intermediate): Building a Multi-Tool Agent

**Task:** Create a more advanced LangChain agent that has access to *multiple* tools and must reason about which one to use for a given task.

> **What is the `agent_scratchpad`?** This is the agent's internal monologue. It's where the agent keeps track of its thought process: which tool it's thinking of using, what the tool's output was, and what it plans to do next. It is an essential part of the prompt for the agent's reasoning process.

**Instructions:**
1.  Keep the `TavilySearchResults` tool from the previous challenge.
2.  Define a new, custom tool for a calculator. You can do this by creating a simple Python function (e.g., `def multiply(a: int, b: int) -> int:`) and then decorating it with the `@tool` decorator from `langchain_core.tools`.
3.  Create a new list of tools that includes both the search tool and your new calculator tool.
4.  Create a new agent and `AgentExecutor` using this expanded list of tools.
5.  Invoke the agent twice with different questions:
    * A question that requires the calculator tool (e.g., "What is 25 * 48?").
    * A question that requires the search tool (e.g., "Who is the current CEO of Apple?").
6.  Observe how the agent correctly chooses which tool to use for each question.

**Expected Quality:** A single agent that can dynamically decide which tool to use based on the user's query, demonstrating the core reasoning capability of an agent.

In [10]:
from langchain_core.tools import tool

# TODO: 1. Define your custom calculator tool
# Use the @tool decorator
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers together."""
    return a * b

# TODO: 2. Create the new list of tools
multi_tool_list = [multiply, search_tool]

# TODO: 3. Create the new multi-tool agent and executor
# The prompt and creation process are the same as before, just with the new tool list.
multi_tool_agent = create_tool_calling_agent(
    llm=llm,
    tools=multi_tool_list,
    prompt=prompt,
)
multi_tool_executor = AgentExecutor(
    agent=multi_tool_agent,
    tools=multi_tool_list,
    return_intermediate_steps=True,
    verbose=True,
)

# TODO: 4. Invoke the agent with a math question
math_question = "What is 25 * 48?"
math_result = multi_tool_executor.invoke({"input": math_question})
print(f"Query: {math_question}\nResult: {math_result}\n")

# TODO: 5. Invoke the agent with a search question
search_question = "Who is the current CEO of Apple?"
search_result = multi_tool_executor.invoke({"input": search_question})
print(f"Query: {search_question}\nResult: {search_result}")



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 25, 'b': 48}`


120025 * 48 = 1200.

> Finished chain.
Query: What is 25 * 48?
Result: {'input': 'What is 25 * 48?', 'output': '25 * 48 = 1200.', 'intermediate_steps': [(ToolAgentAction(tool='multiply', tool_input={'a': 25, 'b': 48}, log="\nInvoking: `multiply` with `{'a': 25, 'b': 48}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_E7cvesbOy4WjlJIA0ftegxLb', 'function': {'arguments': '{"a":25,"b":48}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'service_tier': 'default'}, id='run--c58fc7cf-8a6e-488f-8e0e-7c2cdaeb6921', tool_calls=[{'name': 'multiply', 'args': {'a': 25, 'b': 48}, 'id': 'call_E7cvesbOy4WjlJIA0ftegxLb', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'multiply', 'args': '{"a":25,"b":48}', 'id': 'call_E7cvesbOy4Wjl

### Challenge 3 (Advanced): Improving Tool Selection with Better Docstrings

**Task:** Improve the agent's ability to choose the correct tool by writing more descriptive docstrings.

> **Tip:** The agent doesn't understand your Python code; it only understands the tool's name and its docstring! A clear, descriptive docstring like 'Multiplies two integers together' is critical for helping the agent know when to use the tool.

**Instructions:**
1.  Copy your `multiply` tool from the previous challenge.
2.  Modify its docstring to be much more descriptive. For example: "Use this tool for mathematical calculations involving multiplication. It takes two integers and returns their product."
3.  Re-create your agent and executor with this updated tool.
4.  Invoke the agent again with the math question and observe if its reasoning process (visible in the `verbose=True` output) is more direct.

**Expected Quality:** A demonstration of how prompt engineering the docstring of a tool can significantly improve an agent's performance and reliability.

In [11]:
# TODO: 1. Redefine your custom tool with a more descriptive docstring.
@tool
def better_multiply(a: int, b: int) -> int:
    """ Use this tool for mathematical calculations involving multiplication. It takes two integers and returns their product. """
    return a * b

# TODO: 2. Re-create the tool list, agent, and executor with the improved tool.
improved_tool_list = [better_multiply, search_tool]
improved_agent = create_tool_calling_agent(
    llm=llm,
    tools=improved_tool_list,
    prompt=prompt,
)
improved_executor = AgentExecutor(
    agent=improved_agent,
    tools=improved_tool_list,
    return_intermediate_steps=True,
    verbose=True,
)

# TODO: 3. Invoke the agent again.
math_question = "What is 25 * 48?"
improved_math_result = improved_executor.invoke({"input": math_question})
print(f"Query: {math_question}\nResult: {improved_math_result}")



> Entering new AgentExecutor chain...

Invoking: `better_multiply` with `{'a': 25, 'b': 48}`


120025 multiplied by 48 equals 1200.

> Finished chain.
Query: What is 25 * 48?
Result: {'input': 'What is 25 * 48?', 'output': '25 multiplied by 48 equals 1200.', 'intermediate_steps': [(ToolAgentAction(tool='better_multiply', tool_input={'a': 25, 'b': 48}, log="\nInvoking: `better_multiply` with `{'a': 25, 'b': 48}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_josv2MBQ75QiiXy2ZaZ5ZiWJ', 'function': {'arguments': '{"a":25,"b":48}', 'name': 'better_multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'service_tier': 'default'}, id='run--56c78610-6642-4936-a89d-c6de35514503', tool_calls=[{'name': 'better_multiply', 'args': {'a': 25, 'b': 48}, 'id': 'call_josv2MBQ75QiiXy2ZaZ5ZiWJ', 'type': 'tool_call'}], tool_call_chunks=[{'na

## Lab Conclusion

Congratulations! You have successfully built your first AI agents. You've learned how to give agents tools to extend their capabilities and, most importantly, how to build an agent that can reason about which tool to use for a specific task. This is the foundational skill for all advanced agentic workflows we will explore in the coming days.

> **Key Takeaway:** An agent's ability to reason and act depends on its understanding of its tools. The most critical part of creating a custom tool is writing a clear, descriptive docstring that tells the agent exactly what the tool does and when to use it.